# Visualization of trends detected by Bayesian method

In [1]:
import os, sys

import datetime

import numpy as np
from scipy.stats import norm
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import mplfinance as mpf
import ipywidgets as widgets

In [2]:
%matplotlib widget

In [3]:
df_ge = pd.read_csv("ge.us.txt", engine='python')
df_ge.rename(columns={'Date': 'Time'}, inplace=True)
df_ge['Time'] = pd.to_datetime(df_ge['Time'])

In [4]:
df_ge.tail(5)

,Time,Open,High,Low,Close,Volume,OpenInt
14053,2017-11-06,20.52,20.530,20.08,20.13,60641787,0
14054,2017-11-07,20.17,20.250,20.12,20.21,41622851,0
14055,2017-11-08,20.21,20.320,20.07,20.12,39672190,0
14056,2017-11-09,20.04,20.071,19.85,19.99,50831779,0
14057,2017-11-10,19.98,20.680,19.90,20.49,100698474,0


In [5]:
class BayesModel:
    
    def __init__(self, m, p, trends_interval, sigma):
        self.m = m
        self.p = p
        
        self.K = np.zeros((m, m))

        for i in range(m):
            for j in range(m):
                self.K[i, j] =  1

        self.K = self.K/np.sum(self.K, axis=0)
        self.K = self.p * self.K

        self.K_prime = self.K.copy()

        for i in range(m):
            self.K[i, i] += 1 - self.p

        self.w_prior = np.ones(m)
        self.w_prior = self.w_prior / np.sum(self.w_prior)
        
        self.a = np.linspace(start=-trends_interval, stop=trends_interval, num=m) # different trend sizes
        self.sigma = sigma

    def fit(self, sig):
        
        N = len(sig)
        
        m = self.m

        c = np.zeros((m, N - 1)) # c is a matrix of p(\delta x_k, \ldots, \delta x_n \mid a_j)
        ps = np.zeros(m) # ps is a row of p(\delta_x \mid a_j) for j = 1,...,m 

        w = self.w_prior

        w_traj = np.zeros((m, N))
        w_traj[:, 0] = w

        w_traj_after = np.zeros((m, N))
        w_traj_after[:, 0] = w

        for k in range(1, N):
            # Bayesian recalculation step
            delta_x = sig[k] - sig[k-1]

            for i in range(m):
                w[i] = w[i] * norm.pdf(delta_x, loc=self.a[i], scale=self.sigma)
            w = w / np.sum(w)

            w_traj[:, k] = w

            # markov chain transition due to possible trend   
            w = np.dot(self.K, w)

            w = w / np.sum(w) # norming, to avoid numerical effects due to finite precision

            w_traj_after[:, k] = w

        # find posterior probabilities of trends
        delta_x = sig[N-1] - sig[N-2]
        for j in range(m):            
            c[j, N-2] = norm.pdf(delta_x, loc=self.a[j], scale=self.sigma)
        c[:, N-2] = c[:, N-2] / np.sum(c[:, N-2])

        for i in range(1, N - 1):
            delta_x = sig[N - i - 1] - sig[N - i - 2]
            for j in range(m):
                ps[j] = norm.pdf(delta_x, loc=self.a[j], scale=self.sigma)
            c[:, N-i-2] = np.dot(c[:, N-i-1], self.K) * ps
            c[:, N-i-2] = c[:, N-i-2] / np.sum(c[:, N-i-2])

        self.weights = c * w_traj_after[:, 0: N-1]
        self.weights = self.weights / np.sum(self.weights, axis=0)

In [6]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

class BayesTrendsPlotter(widgets.HBox):         
    def __init__(self, df, fsym, tsym, exchange):
        super().__init__()
        
        self.fsym = fsym
        self.tsym = tsym
        self.exchange = exchange
        
        self.df_raw = df
        self.df_raw.index = self.df_raw['Time']

        output = widgets.Output()
        
        with output:
            self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(8, 8))
         
        self.fig.canvas.toolbar_position = 'bottom'
 
        # define widgets
        self.curr_point_slider = widgets.IntSlider(value=100, min=0, max=len(self.df_raw)-1, step=1, 
                                                 description='point number')
        self.delta_left_slider = widgets.IntSlider(value=80, min=0, max=1200, step=1, 
                                              description='left points')
        self.delta_right_slider = widgets.IntSlider(value=20, min=0, max=400, step=1, 
                                              description='right points')

        self.button_plus_one = widgets.Button(description='+1')
        self.button_plus_five = widgets.Button(description='+5')
        self.button_plus_ten = widgets.Button(description='+10')
        self.button_minus_one = widgets.Button(description='-1')
        self.button_minus_five = widgets.Button(description='-5')
        self.button_minus_ten = widgets.Button(description='-10')
        
        self.p_text = widgets.FloatText(value=0.1, description='p')
        
        self.trends_interval_text = widgets.FloatText(value=100, description='t_interval')
        self.sigma_text = widgets.FloatText(value=100, description='noise sigma')
        
        self.num_trends_slider = widgets.IntSlider(value=3, min=3, max=20, step=1, description='num trends')
        
        self.trend_to_show_slider = widgets.IntSlider(value=2, min=0, max=2, step=1, description='show trend')
        
        self.trend_size_text= widgets.FloatText(value=100, description='trend size')

        controls = widgets.VBox([self.curr_point_slider, 
                                 self.delta_left_slider,
                                 self.delta_right_slider,
                                 self.button_plus_one,
                                 self.button_plus_five, 
                                 self.button_plus_ten,
                                 self.button_minus_one,
                                 self.button_minus_five,
                                 self.button_minus_ten,
                                 self.p_text,
                                 self.trends_interval_text,
                                 self.sigma_text,
                                 self.num_trends_slider,
                                 self.trend_to_show_slider, 
                                 self.trend_size_text])
 
        controls.layout = make_box_layout()
         
        out_box = widgets.Box([output])
        output.layout = make_box_layout()
        
        with output:
            self.plot_bayes_trends(curr_point_num=100, delta_left=80, delta_right=20, 
                                   p=0.1, num_trends=3, trend_num_to_show=2, trends_interval=100, sigma=100)
 
        # observe stuff
        self.curr_point_slider.observe(self.update_all)
        self.delta_left_slider.observe(self.update_all)
        self.delta_right_slider.observe(self.update_all)

        self.button_plus_one.on_click(self.on_button_plus_one_clicked)
        self.button_plus_five.on_click(self.on_button_plus_five_clicked)
        self.button_plus_ten.on_click(self.on_button_plus_ten_clicked)
        self.button_minus_one.on_click(self.on_button_minus_one_clicked)
        self.button_minus_five.on_click(self.on_button_minus_five_clicked)
        self.button_minus_ten.on_click(self.on_button_minus_ten_clicked)
        
        # self.p_slider.observe(self.update_all)
        self.p_text.observe(self.update_all)
        self.num_trends_slider.observe(self.update_all)
        self.trend_to_show_slider.observe(self.update_all)
        self.trends_interval_text.observe(self.update_all)
        self.sigma_text.observe(self.update_all)
        
        self.curr_point_slider.value = 100
        self.delta_left_slider.value = 80
        self.delta_right_slider.value = 20
 
        # add to children
        self.children = [controls, output]

    def plot_bayes_trends(self, curr_point_num, delta_left, delta_right, p, num_trends, trend_num_to_show, trends_interval, sigma):
        
        self.ax1.clear()
        self.ax2.clear()
        
        self.trend_to_show_slider.max = num_trends - 1
        
        start = max(curr_point_num-delta_left, 0)
        end = min(curr_point_num+delta_right, len(self.df_raw)-1)

        mpf.plot(self.df_raw[start: end+1], ax=self.ax1, type='candle', style='charles')
            
        self.ax1.axvline(x=curr_point_num-start, color='blue')

        quantities = self.df_raw['Close'].values[start: end + 1]
            
        m = num_trends            
        bayes_model = BayesModel(m, p, trends_interval, sigma) 
        
        self.trend_size_text.value = bayes_model.a[trend_num_to_show]
            
        bayes_model.fit(quantities)
            
        weights = bayes_model.weights
        
        current_weights = weights[trend_num_to_show, :]
        
        self.ax1.set_title(f"{self.fsym}/{self.tsym} on {self.exchange}")

        self.ax1.grid()
        
        self.ax2.set_ylim(-0.1, 1.1)
        
        self.ax2.plot(current_weights)
        
        self.ax2.set_title(f"Probabilities of trend {trend_num_to_show}")
        
        self.ax2.grid()
        
    # callback functions
    def update_all(self, change):        
        self.plot_bayes_trends(curr_point_num=self.curr_point_slider.value,
                               delta_left=self.delta_left_slider.value, 
                               delta_right=self.delta_right_slider.value,
                               p=self.p_text.value, # self.p_slider.value,
                               num_trends=self.num_trends_slider.value,
                               trend_num_to_show=self.trend_to_show_slider.value, 
                               trends_interval=self.trends_interval_text.value,
                               sigma=self.sigma_text.value)
        
    def on_button_plus_one_clicked(self, _):
        if self.curr_point_slider.value + 1 < len(self.df_raw):
            self.curr_point_slider.value += 1
        self.plot_bayes_trends(curr_point_num=self.curr_point_slider.value, 
                               delta_left=self.delta_left_slider.value, 
                               delta_right=self.delta_right_slider.value,
                               p=self.p_text.value, #self.p_slider.value,
                               num_trends=self.num_trends_slider.value,
                               trend_num_to_show=self.trend_to_show_slider.value,
                               trends_interval=self.trends_interval_text.value,
                               sigma=self.sigma_text.value)
        
    def on_button_plus_five_clicked(self, _):
        if self.curr_point_slider.value + 5 < len(self.df_raw):
            self.curr_point_slider.value += 5
        self.plot_bayes_trends(curr_poinself.p_text.value, #t_num=self.curr_point_slider.value, 
                               delta_left=self.delta_left_slider.value, 
                               delta_right=self.delta_right_slider.value,
                               p=self.p_text.value, #self.p_slider.value,
                               num_trends=self.num_trends_slider.value,
                               trend_num_to_show=self.trend_to_show_slider.value,
                               trends_interval=self.trends_interval_text.value,
                               sigma=self.sigma_text.value)
        
    def on_button_plus_ten_clicked(self, _):
        if self.curr_point_slider.value + 10 < len(self.df_raw):
            self.curr_point_slider.value += 10
        self.plot_bayes_trends(curr_point_num=self.curr_point_slider.value, 
                               delta_left=self.delta_left_slider.value, 
                               delta_right=self.delta_right_slider.value,
                               p=self.p_text.value, #self.p_slider.value,
                               num_trends=self.num_trends_slider.value,
                               trend_num_to_show=self.trend_to_show_slider.value,
                               trends_interval=self.trends_interval_text.value, 
                               sigma=self.sigma_text.value)
        
    def on_button_minus_one_clicked(self, _):
        if self.curr_point_slider.value - 1 >= 0:
            self.curr_point_slider.value -= 1
        self.plot_bayes_trends(curr_point_num=self.curr_point_slider.value, 
                               delta_left=self.delta_left_slider.value, 
                               delta_right=self.delta_right_slider.value,
                               p=self.p_text.value, #self.p_slider.value,
                               num_trends=self.num_trends_slider.value, 
                               trend_num_to_show=self.trend_to_show_slider.value,
                               trends_interval=self.trends_interval_text.value, 
                               sigma=self.sigma_text.value)
        
    def on_button_minus_five_clicked(self, _):
        if self.curr_point_slider.value - 5 >= 0:
            self.curr_point_slider.value -= 5
        self.plot_bayes_trends(curr_point_num=self.curr_point_slider.value, 
                               delta_left=self.delta_left_slider.value, 
                               delta_right=self.delta_right_slider.value,
                               p=self.p_text.value, #self.p_slider.value,
                               num_trends=self.num_trends_slider.value, 
                               trend_num_to_show=self.trend_to_show_slider.value,
                               trends_interval=self.trends_interval_text.value,
                               sigma=self.sigma_text.value)
        
    def on_button_minus_ten_clicked(self, _):
        if self.curr_point_slider.value - 10 >= 0:
            self.curr_point_slider.value -= 10
        self.plot_bayes_trends(curr_point_num=self.curr_point_slider.value, 
                               delta_left=self.delta_left_slider.value, 
                               delta_right=self.delta_right_slider.value,
                               p=self.p_text.value, #self.p_slider.value,
                               num_trends=self.num_trends_slider.value,
                               trend_num_to_show=self.trend_to_show_slider.value,
                               trends_interval=self.trends_interval_text.value,
                               sigma=self.sigma_text.value)         

In [7]:
BayesTrendsPlotter(df_ge, 'BTC', 'USDT', 'Binance')

BayesTrendsPlotter(children=(VBox(children=(IntSlider(value=100, description='point number', max=14057), IntSl…